In [ ]:
# 패키지 설치
!pip install transformers accelerate torch flask pyngrok nest-asyncio -q
print('[OK] Packages installed')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata

MODEL_NAME = "Qwen/Qwen2-1.5B-Instruct"

print(f"Loading model: {MODEL_NAME}...")

try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("[OK] HuggingFace token loaded from secrets")
except:
    HF_TOKEN = "YOUR_HF_TOKEN_HERE"
    print("[WARNING] Using hardcoded token")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=HF_TOKEN,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    token=HF_TOKEN,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"[OK] Model loaded on {model.device}")
print(f"[OK] Model: {MODEL_NAME}")

In [ ]:
from flask import Flask, request, jsonify
import traceback

app = Flask(__name__)


@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        "status": "healthy",
        "model": MODEL_NAME
    })


@app.route('/summarize', methods=['POST'])
def summarize_analysis():
    """
    CAMEO analysis result summary in ENGLISH
    (Translation will be handled by external API)
    """
    try:
        print("=" * 70)
        print("[Summarize] New request")
        print("=" * 70)

        data = request.get_json()

        if not data or "analysis" not in data:
            return jsonify({
                "success": False,
                "error": "Missing 'analysis' field"
            }), 400

        analysis = data["analysis"]
        summary = analysis.get("summary", {})
        dangerous_pairs = analysis.get("dangerous_pairs", [])
        caution_pairs = analysis.get("caution_pairs", [])

        print(f"[Summarize] Received: {summary.get('total_pairs', 0)} pairs")

        # STRONG English-only prompt
        prompt = f"""You are a chemical safety expert. Write ONLY IN ENGLISH. DO NOT use Korean or any other language.

Analysis Results:
- Total combinations tested: {summary.get('total_pairs', 0)}
- Dangerous combinations: {summary.get('dangerous_count', 0)}
- Caution required: {summary.get('caution_count', 0)}
- Safe combinations: {summary.get('safe_count', 0)}
- Overall status: {summary.get('overall_status', 'Unknown')}

"""

        if dangerous_pairs:
            prompt += "\nDangerous Combinations Found:\n"
            for i, pair in enumerate(dangerous_pairs[:3], 1):
                chem1 = pair.get('chemical_1', 'Unknown')
                chem2 = pair.get('chemical_2', 'Unknown')
                prompt += f"\n{i}. {chem1} + {chem2}\n"
                prompt += f"   Status: {pair.get('status', 'Unknown')}\n"
                prompt += f"   Severity Score: {pair.get('severity_score', 0)}/20\n"
                hazards = pair.get('hazards', [])
                if hazards:
                    prompt += f"   Key Hazards: {hazards[0][:100]}...\n"

        if caution_pairs:
            prompt += f"\nCaution Combinations: {len(caution_pairs)} found\n"

        if summary.get('safe_count', 0) > 0:
            prompt += f"\nSafe Combinations: {summary.get('safe_count')} found\n"

        prompt += """
IMPORTANT: Write your response ONLY IN ENGLISH. Do not use Korean.

Write a clear, professional summary in English (3-5 sentences) covering:
1. Overall risk assessment
2. Main hazards and specific dangerous combinations
3. Critical safety recommendations

Use proper chemical terminology. Be concise and accurate.

Response (ENGLISH ONLY):"""

        print(f"[Summarize] Generating English summary...")

        messages = [{"role": "user", "content": prompt}]

        prompt_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        inputs = tokenizer(
            prompt_text,
            return_tensors="pt",
            truncation=True,
            max_length=1536
        )

        if model.device.type != "cpu":
            inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=400,
                temperature=0.3,
                top_p=0.9,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        result = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Remove prompt - find where the actual response starts
        markers = [
            "Response (ENGLISH ONLY):",
            "Be concise and accurate.",
            "assistant",
            "Assistant:",
            "<|im_start|>assistant",
            "ENGLISH ONLY):"
        ]

        for marker in markers:
            if marker in result:
                result = result.split(marker)[-1].strip()

        # Clean up
        result = result.strip(". \t\n")

        # Remove empty lines - FIXED: use \n not empty string
        lines = [line.strip() for line in result.split("\n") if line.strip()]
        result = " ".join(lines)

        # Remove any Korean characters (fallback safety)
        import re
        korean_pattern = re.compile('[가-힣]+')
        if korean_pattern.search(result):
            print("[WARNING] Korean detected in output, attempting to filter...")
            # If Korean is found, try to extract only English portions
            sentences = result.split('.')
            english_sentences = []
            for sent in sentences:
                if not korean_pattern.search(sent):
                    english_sentences.append(sent)
            if english_sentences:
                result = '. '.join(english_sentences).strip() + '.'

        print(f"[Summarize] Generated: {len(result)} chars (English)")
        print(f"[Summarize] Preview: {result[:100]}...")

        return jsonify({
            "success": True,
            "summary": result,
            "language": "en"
        })

    except Exception as e:
        print(f"[ERROR] {str(e)}")
        traceback.print_exc()

        return jsonify({
            "success": False,
            "error": str(e),
            "traceback": traceback.format_exc()
        }), 500


print("[OK] Flask app configured")
print("[OK] Endpoints: GET /health, POST /summarize (English output)")


In [ ]:
from pyngrok import ngrok
import nest_asyncio

nest_asyncio.apply()

NGROK_AUTH_TOKEN = "34dflI9kRYLX8COEWV7CxYAAQMA_W7dBiZTCfp6oe3Lf1LTY"

print("=" * 70)
print("Starting ngrok tunnel...")
print("=" * 70)

try:
    ngrok.kill()
except:
    pass

try:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    public_url = ngrok.connect(5000, bind_tls=True)
    
    if hasattr(public_url, 'public_url'):
        tunnel_url = public_url.public_url
    else:
        tunnel_url = str(public_url)
    
    print("\n" + "=" * 70)
    print("[OK] API Server Ready!")
    print("=" * 70)
    print(f"\nPublic URL: {tunnel_url}")
    print(f"\n사용 방법:")
    print(f"  1. 위 URL 복사")
    print(f"  2. 로컬 .env 파일에 추가:")
    print(f"     COLAB_API_URL={tunnel_url}")
    print(f"\n테스트:")
    print(f"  curl {tunnel_url}/health")
    print("\n" + "=" * 70 + "\n")
    
    print("Starting Flask server on port 5000...")
    print("Endpoints:")
    print("  GET  /health")
    print("  POST /summarize")
    print("=" * 70 + "\n")
    
    app.run(port=5000)
    
except Exception as e:
    print(f"\n[ERROR] {e}")
    traceback.print_exc()